## Load Resume Dataset from Kaggle
https://www.kaggle.com/datasets/snehaanbhawal/resume-dataset/data

In [2]:
# load data
import pandas as pd
df = pd.read_csv("Resume.csv")
print(df.head())

         ID                                         Resume_str  \
0  16852973           HR ADMINISTRATOR/MARKETING ASSOCIATE\...   
1  22323967           HR SPECIALIST, US HR OPERATIONS      ...   
2  33176873           HR DIRECTOR       Summary      Over 2...   
3  27018550           HR SPECIALIST       Summary    Dedica...   
4  17812897           HR MANAGER         Skill Highlights  ...   

                                         Resume_html Category  
0  <div class="fontsize fontface vmargins hmargin...       HR  
1  <div class="fontsize fontface vmargins hmargin...       HR  
2  <div class="fontsize fontface vmargins hmargin...       HR  
3  <div class="fontsize fontface vmargins hmargin...       HR  
4  <div class="fontsize fontface vmargins hmargin...       HR  


## Preprocessing 

In [3]:
df.shape

(2484, 4)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2484 entries, 0 to 2483
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ID           2484 non-null   int64 
 1   Resume_str   2484 non-null   object
 2   Resume_html  2484 non-null   object
 3   Category     2484 non-null   object
dtypes: int64(1), object(3)
memory usage: 77.8+ KB


In [5]:
df['Category'].unique()

array(['HR', 'DESIGNER', 'INFORMATION-TECHNOLOGY', 'TEACHER', 'ADVOCATE',
       'BUSINESS-DEVELOPMENT', 'HEALTHCARE', 'FITNESS', 'AGRICULTURE',
       'BPO', 'SALES', 'CONSULTANT', 'DIGITAL-MEDIA', 'AUTOMOBILE',
       'CHEF', 'FINANCE', 'APPAREL', 'ENGINEERING', 'ACCOUNTANT',
       'CONSTRUCTION', 'PUBLIC-RELATIONS', 'BANKING', 'ARTS', 'AVIATION'],
      dtype=object)

In [6]:
df['Category'].value_counts()

Category
INFORMATION-TECHNOLOGY    120
BUSINESS-DEVELOPMENT      120
ADVOCATE                  118
CHEF                      118
ENGINEERING               118
ACCOUNTANT                118
FINANCE                   118
FITNESS                   117
AVIATION                  117
SALES                     116
BANKING                   115
HEALTHCARE                115
CONSULTANT                115
CONSTRUCTION              112
PUBLIC-RELATIONS          111
HR                        110
DESIGNER                  107
ARTS                      103
TEACHER                   102
APPAREL                    97
DIGITAL-MEDIA              96
AGRICULTURE                63
AUTOMOBILE                 36
BPO                        22
Name: count, dtype: int64

In [7]:
# drop column 'Resume_html'
df = df.drop(columns=["Resume_html"])

In [8]:
# show full column width
pd.set_option('display.max_colwidth', None)
# show first row
df["Resume_str"].iloc[0]

"         HR ADMINISTRATOR/MARKETING ASSOCIATE\n\nHR ADMINISTRATOR       Summary     Dedicated Customer Service Manager with 15+ years of experience in Hospitality and Customer Service Management.   Respected builder and leader of customer-focused teams; strives to instill a shared, enthusiastic commitment to customer service.         Highlights         Focused on customer satisfaction  Team management  Marketing savvy  Conflict resolution techniques     Training and development  Skilled multi-tasker  Client relations specialist           Accomplishments      Missouri DOT Supervisor Training Certification  Certified by IHG in Customer Loyalty and Marketing by Segment   Hilton Worldwide General Manager Training Certification  Accomplished Trainer for cross server hospitality systems such as    Hilton OnQ  ,   Micros    Opera PMS   , Fidelio    OPERA    Reservation System (ORS) ,   Holidex    Completed courses and seminars in customer service, sales strategies, inventory control, loss pr

In [15]:
import re

# Create a boolean column that marks resumes containing HTML tags
df["has_html"] = df["Resume_str"].apply(lambda x: bool(re.search(r"<.*?>", str(x))))

# Count how many resumes have HTML tags
print("Resumes with HTML tags:", df["has_html"].sum())
print("Resumes without HTML tags:", len(df) - df["has_html"].sum())



Resumes with HTML tags: 10
Resumes without HTML tags: 2474


In [16]:
# remove extra spaces, line breaks and html tags and add clean text as a column (Resume_str = Resume_clean : just without the extra)
import re

def clean_text(text):
    text = re.sub(r'<.*?>', ' ', str(text)) 
    text = re.sub(r'\s+', ' ', text) 
    return text.strip()

df["Resume_clean"] = df["Resume_str"].apply(clean_text)
df["Resume_clean"].head(2)


0                                                                                                                                                      HR ADMINISTRATOR/MARKETING ASSOCIATE HR ADMINISTRATOR Summary Dedicated Customer Service Manager with 15+ years of experience in Hospitality and Customer Service Management. Respected builder and leader of customer-focused teams; strives to instill a shared, enthusiastic commitment to customer service. Highlights Focused on customer satisfaction Team management Marketing savvy Conflict resolution techniques Training and development Skilled multi-tasker Client relations specialist Accomplishments Missouri DOT Supervisor Training Certification Certified by IHG in Customer Loyalty and Marketing by Segment Hilton Worldwide General Manager Training Certification Accomplished Trainer for cross server hospitality systems such as Hilton OnQ , Micros Opera PMS , Fidelio OPERA Reservation System (ORS) , Holidex Completed courses and seminars in cu

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2484 entries, 0 to 2483
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   ID            2484 non-null   int64 
 1   Resume_str    2484 non-null   object
 2   Category      2484 non-null   object
 3   Resume_clean  2484 non-null   object
 4   has_html      2484 non-null   bool  
dtypes: bool(1), int64(1), object(3)
memory usage: 80.2+ KB


In [18]:
df= df.drop(columns=['has_html'])

### Entity Extraction (Use Case: Skill, Experience,Education)

In [21]:
# Information Extraction Prompt Generator
def create_extraction_prompt(resume_text):
    prompt = f"""
You are an experienced HR specialist. Extract the following information from the candidate's resume:

- Skills
- Experiences (with years if mentioned)
- Certifications
- Education

Resume:
{resume_text}

Respond ONLY in JSON format like this:
{{
  "skills": [],
  "experiences": [],
  "certifications": [],
  "education": []
}}
"""
    return prompt.strip()

# Prompt Dataset
df["extraction_prompt"] = df["Resume_clean"].apply(create_extraction_prompt)

# Preview
df[["Category", "extraction_prompt"]].head(3)


Category  \
0       HR   
1       HR   
2       HR   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2484 entries, 0 to 2483
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   ID                 2484 non-null   int64 
 1   Resume_str         2484 non-null   object
 2   Category           2484 non-null   object
 3   Resume_clean       2484 non-null   object
 4   extraction_prompt  2484 non-null   object
dtypes: int64(1), object(4)
memory usage: 97.2+ KB


In [23]:
# Sample ca. 100 resumes, by category 5 from each (proportional)
df_sample = df.groupby("Category", group_keys=False).apply(lambda x: x.sample(min(len(x), 5), random_state=42))
df_sample.shape


C:\Users\semra\AppData\Local\Temp\ipykernel_9620\2823696182.py:2: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_sample = df.groupby("Category", group_keys=False).apply(lambda x: x.sample(min(len(x), 5), random_state=42))


(120, 5)

In [25]:
df_sample.to_csv("resume_sample.csv", index=False)
df_sample.info()

<class 'pandas.core.frame.DataFrame'>
Index: 120 entries, 1864 to 379
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   ID                 120 non-null    int64 
 1   Resume_str         120 non-null    object
 2   Category           120 non-null    object
 3   Resume_clean       120 non-null    object
 4   extraction_prompt  120 non-null    object
dtypes: int64(1), object(4)
memory usage: 9.7+ KB


In [ ]:
# gather output from LLM

In [ ]:
# annotate a sample ( 5 categories, 3-4 per category, 15-20 annotated resumes)